# Hyper-parameter Optimization

In this notebook hyper-parameters of XGBoost are optimized for [HIGGS](https://archive.ics.uci.edu/ml/datasets/HIGGS) dataset.

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID

env: CUDA_DEVICE_ORDER=PCI_BUS_ID


In [2]:
import tensorflow as tf
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.1)
s = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

import numpy as np

Here, [scikit-optimize](https://scikit-optimize.github.io/) (skopt) library is used.

In [4]:
from skopt import Optimizer

from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import Matern, RBF, WhiteKernel

from skopt.learning import RandomForestRegressor

from skopt.acquisition import gaussian_ei as acq_func

/mnt/mlhep2018/pyenv/versions/3.6.6/envs/mlhep/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Loading data

The original dataset is quite large, thus, in order to save time, only a portion of it is used.

In [5]:
DATA_ROOT='/mnt/mlhep2018/datasets/'

SAMPLE_LIMIT=100000

In [6]:
import os.path as osp

data = np.genfromtxt(
    osp.join(DATA_ROOT, 'HIGGS.csv'),
    dtype='float32',
    delimiter=',',
    max_rows=SAMPLE_LIMIT
)

X, y = data[:, 1:], data[:, 0]

In [7]:
X.shape

(100000, 28)

In [8]:
### again to save training time only 10% of the loaded data is used for training.
from sklearn.model_selection import train_test_split
# want to split data set into three sections
# one for training hyperparameters
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

## Max depth tuning

For illustration purposes we perform Baysian Optimization on 1 parameter: `learning rate`.

Learning rate in XGBoost can vary dramatically by orders of magnitude, thus, we use *logarithmic scale*.

The goal is to maximize `ROC AUC` (minimize 1 - `ROC AUC`).

In [ ]:
def target_function_1(log_learning_rate, X_train=X_train, y_train=y_train, X_score=X_test, y_score=y_test):
    from xgboost import XGBClassifier
    from sklearn.metrics import roc_auc_score
    
    xgb = XGBClassifier(
        max_depth=3, 
        n_estimators=50,
        learning_rate=np.exp(log_learning_rate),
        objective='binary:logistic',
        n_jobs=20,
        reg_lambda=0.0
    )

    xgb.fit(X_train, y_train)
    predictions = xgb.predict_proba(X_score)[:, 1]
    
    return 1 - roc_auc_score(y_score, predictions)

In [ ]:
### learning rate <- [1.0e-3, 1]
dimensions_1 = [
    (np.log(1.0e-3), 0)
]

In [ ]:
%%time

target_function_1(0.0, X_train, y_train, X_val, y_val)

In [ ]:
### utility function to inspect status of BO.

def plot_bo(bo, suggestion=None, value=None):
    a, b = bo.space.bounds[0]
    
    ### getting the latest model
    model = bo.models[-1]
    
    xs = np.linspace(a, b, num=100)
    x_model = bo.space.transform(xs.reshape(-1, 1).tolist())
    
    mean, std = model.predict(x_model, return_std=True)
    
    plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.scatter(
        np.array(bo.Xi)[:, 0],
        np.array(bo.yi),
        color='red',
        label='observations'
    )
    if suggestion is not None:
        plt.scatter([suggestion], value, color='blue', label='suggestion')
    
    plt.plot(xs, mean, color='green', label='model')
    plt.fill_between(xs, mean - 1.96 * std, mean + 1.96 * std, alpha=0.1, color='green')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    acq = acq_func(x_model, model, np.min(bo.yi))
    plt.plot(xs, acq, label='Expected Improvement')
    plt.legend()
    
    plt.show()

In [ ]:
### another utility function
def cum_min(xs):
    result = np.zeros_like(xs)
    cmin = xs[0]
    
    result[0] = xs[0]
    
    for i in range(1, xs.shape[0]):
        if cmin > xs[i]:
            cmin = xs[i]

        result[i] = cmin
    
    return result

In [ ]:
from IPython import display

### plots progress of BO over time.

def plot_convergence(bo):
    display.clear_output(wait=True)
    values = np.array(bo.yi)
    
    plt.figure(figsize=(12, 6))
    plt.plot(cum_min(values), label='minimal discovered')
    plt.scatter(np.arange(len(bo.yi)), bo.yi, label='observations')
    plt.xlabel('step', fontsize=14)
    plt.ylabel('loss', fontsize=14)
    
    plt.legend(loc='upper right', fontsize=18)
    
    plt.show()

### Bayesian optimization with Gaussian Process

In [ ]:
### create an instance of Baysian Optimizer
bo_gp_1 = Optimizer(
    ### telling optimizer boundaries for each parameter
    dimensions=dimensions_1,
    
    ### setting regressor
    base_estimator=GaussianProcessRegressor(
        kernel=RBF(length_scale_bounds=[1.0e-6, 1.0e+6]) + \
            WhiteKernel(noise_level=1.0e-5, noise_level_bounds=[1.0e-6, 1.0e-2]),
    ),
    n_initial_points=2,
    acq_func='EI',   
)

In [ ]:
for i in range(20):
    x = bo_gp_1.ask()
    f = target_function_1(x[0], X_train, y_train, X_val, y_val)

    if len(bo_gp_1.models) > 0:
        plot_bo(bo_gp_1, suggestion=x, value=f)
    
    bo_gp_1.tell(x, f)

In [ ]:
plot_convergence(bo_gp_1)

### Final test

In [ ]:
best = np.argmin(bo_gp_1.yi)
best_parameters = bo_gp_1.Xi[best]

print('Best log learning rate: %.2e' % best_parameters[0])

In [ ]:
best_auc = 1 - target_function_1(best_parameters[0], X_train, y_train, X_test, y_test)

print('Best ROC AUC: %.3lf' % best_auc)

### Bayesian optimization with Random Forest

In [ ]:
### the same but with random forest

bo_rf_1 = Optimizer(
    dimensions=dimensions_1,
    base_estimator=RandomForestRegressor(
        n_estimators=100,
        min_variance=1.0e-6
    ),
    n_initial_points=3,
    acq_func='EI',   
)

In [ ]:
for i in range(20):
    x = bo_rf_1.ask()
    f = target_function_1(x[0])

    if len(bo_rf_1.models) > 0:
        plot_bo(bo_rf_1, suggestion=x, value=f)
    
    bo_rf_1.tell(x, f)

In [ ]:
plot_convergence(bo_rf_1)

In [ ]:
best = np.argmin(bo_rf_1.yi)
best_parameters = bo_rf_1.Xi[best]

print('Best log learning rate: %.2e' % best_parameters[0])

In [ ]:
best_auc = 1 - target_function_1(best_parameters[0], X_train, y_train, X_test, y_test)

print('Best ROC AUC: %.3lf' % best_auc)

## Tuning all important hyper-parameters simultaneously

In [ ]:
def target_function_4(params, X_train, y_train, X_score, y_score):
    """
    params - array with 4 values: [max tree depth, number of estimators, log learning rate, reg_lambda]
    X_train, y_train - training dataset;
    X_score, y_score - dataset for evaluating quality of the classifier;
    
    Returns 1 - ROC AUC evaluated on `X_score` and `y_score`. 
    """
    max_depth, n_estimators, log_learning_rate, reg_lambda = params

    max_depth = int(np.ceil(max_depth))
    n_estimators = int(np.ceil(n_estimators))
    
    learning_rate = np.exp(log_learning_rate)

    from xgboost import XGBClassifier
    from sklearn.metrics import roc_auc_score
    
    xgb = XGBClassifier(
        max_depth=max_depth,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        objective='binary:logistic',
        n_jobs=4
    )
    xgb.fit(X_train, y_train)
    predictions = xgb.predict_proba(X_score)[:, 1]
    
    return 1 - roc_auc_score(y_score, predictions)

In [ ]:
def print_best(bo):
    best_result_index = np.argmin(bo.yi)
    best_parameters = bo.Xi[best_result_index]

    max_depth, n_estimators, log_learning_rate, reg_lambda = best_parameters
    
    print(
        'Best model:\n\nmax_depth=%d\nn_estimators=%d\nlearning_rate=%.1e\nreg_lambda=%.3lf' % (
            int(np.ceil(max_depth)),
            int(np.ceil(n_estimators)),
            np.exp(log_learning_rate),
            reg_lambda
        )
    )
    
    best_auc = target_function_4(best_parameters, X_train, y_train, X_test, y_test)
    print('Best ROC AUC: %.3lf' % (1 - best_auc))

In [ ]:
dimensions_4 =[
    ### max_depth
    (1.0, 20.0),
    
    ### n_estimators
    (1.0, 500.0),
    
    ### log_learning rate
    (np.log(1.0e-3), np.log(1.0)),
    
    ### l2 reg
    (0.0, 1.0)
]

#### Bayesian Optimization with Gaussian Process

In [ ]:
bo_gp_4 = Optimizer(
    dimensions=dimensions_4,
    base_estimator=GaussianProcessRegressor(
        kernel=RBF(length_scale_bounds=[1.0e-3, 1.0e+3]) +\
            WhiteKernel(noise_level=1.0e-5, noise_level_bounds=[1.0e-6, 1.0e-2])
    ),
    n_initial_points=5,
    acq_func='EI',   
)

In [ ]:
for i in range(50):
    x = bo_gp_4.ask()
    f = target_function_4(x, X_train, y_train, X_val, y_val)
    
    bo_gp_4.tell(x, f)
    
    plot_convergence(bo_gp_4)

In [ ]:
print_best(bo_gp_4)

#### The same with Random forest

In [ ]:
bo_rf_4 = Optimizer(
    dimensions=dimensions_4,
    base_estimator=RandomForestRegressor(
        n_estimators=100, n_jobs=4, min_variance=1.0e-6
    ),
    n_initial_points=5,
    acq_func='EI',   
)

In [ ]:
for i in range(50):
    x = bo_rf_4.ask()
    f = target_function_4(x, X_train, y_train, X_val, y_val)
    
    bo_rf_4.tell(x, f)
    
    plot_convergence(bo_rf_4)

In [ ]:
print_best(bo_rf_4)

### Structural Optimization


Sometimes it is preferable to have a good classifier with a particular configuration.

For example, smaller ensemble is preferable to large ones (given the same quality), since small ensemble are faster to compute. One way to find preferable classifiers is to introduce a penalty to the target function:

$$\mathcal{L} = \mathrm{quality\;metric} + \mathrm{penalty}$$


For example, number of CPU operations per prediction can introduced as a penalty: 
$$\mathcal{L} = \mathrm{ROC\; AUC} + C \cdot \mathrm{tree\;depth} \cdot \mathrm{ensemble\;size}$$

In [ ]:
def target_function_4_struct(params, X_train, y_train, X_score, y_score):
    """
    params - array with 4 values: [max tree depth, number of estimators, log learning rate, reg_lambda]
    X_train, y_train - training dataset;
    X_score, y_score - dataset for evaluating quality of the classifier;
    
    Returns 1 - ROC AUC evaluated on `X_score` and `y_score`. 
    """
    max_depth, n_estimators, log_learning_rate, reg_lambda = params

    max_depth = int(np.ceil(max_depth))
    n_estimators = int(np.ceil(n_estimators))
    
    learning_rate = np.exp(log_learning_rate)

    from xgboost import XGBClassifier
    from sklearn.metrics import roc_auc_score
    
    xgb = XGBClassifier(
        max_depth=max_depth,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        objective='binary:logistic',
        n_jobs=4
    )
    xgb.fit(X_train, y_train)
    predictions = xgb.predict_proba(X_score)[:, 1]
    
    score = 1 - roc_auc_score(y_score, predictions)
    
    penalty = 5.0e-5 * n_estimators * max_depth
    
    return score + penalty 

In [ ]:
bo_gp_4_struct = Optimizer(
    dimensions=dimensions_4,
    base_estimator=GaussianProcessRegressor(
        kernel=RBF(length_scale_bounds=[1.0e-3, 1.0e+3]) +\
            WhiteKernel(noise_level=1.0e-5, noise_level_bounds=[1.0e-6, 1.0e-2])
    ),
    n_initial_points=5,
    acq_func='EI',   
)

In [ ]:
for i in range(100):
    x = bo_gp_4_struct.ask()
    f = target_function_4_struct(x, X_train, y_train, X_val, y_val)
    
    bo_gp_4_struct.tell(x, f)
    
    plot_convergence(bo_gp_4_struct)

In [ ]:
print_best(bo_gp_4_struct)

## Your turn

Try to optimize neural network architecture for the HIGGS dataset.

Your neural network **must** have less than $10^4$ parameters!

Target metric: $\mathrm{ROC\;AUC} \to \max$

**Normal problem**:
- fix number of layers (e.g. 5);
- choose type of the layers, non-linearity etc (e.g. `Dense(activation='relu')`);
- optimize number of units in each layer;
- introduce restrictions on number of parameters, either by:
    - penalty e.g. `C * max(num_parameters - 1.0e+4, 0)`;
    - scaling number of units until number of parameters drops to $10^4$;
    - using fractions of total number of parameters for each layer (see `softmax` trick);
- you may want to also include regularization coefficients (e.g. `alpha * l1 + beta * l2`).

**Mad-data-scientist problem**:
- try to optimize everything:
    - find a way to encode variable number of layers (e.g. by introducing limit on number of layers);
    - find a way to optimize activation function:
        - e.g. by preselecting small set of activation functions and taking linear combination of them;
        - skopt alse can deal with descrite variables (please, refer to the documentation of `dimensions` argument for optimizer);
        - alternatively, you can introduce coeficients for each activation function (select 3-4) for each layers and split your layers according to these coefficients: `Concat(Dense(total_units * coef[0], activation='relu'), Dense(total_units * coef[1], activation='sigmoid'))`.

**You can evaluate on the test set only once!**

Tips:
- try to optimize 1 parameter first, e.g. to select an appropriate kernel for GP, estimate noise level;
-  `sigmoid` and `tanh` does not differ much (there is an easy-to-prove procedure that replaces one with another w/o changing the network);
- if your want to have parameters that sum to one use `softmax` and fix one of the parameters to `1` (to remove overparametrization);
- BO does not handle very well ordered descrete variables, it might be better to use `int(param)`:
    - you might want to use cache in this case.
- use small portion of the dataset e.g. `10^4` examples to accelerate search;
- if you have symmetries in your parameterization, i.e. N parameter values correspond to the same network, you can report to BO all N parameters with the same value.

In [9]:
### don't forget about others!

import tensorflow as tf
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.1)
tf_session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

/mnt/mlhep2018/pyenv/versions/3.6.6/envs/mlhep/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [10]:
import keras

keras.backend.tensorflow_backend.set_session(tf_session)

Using TensorFlow backend.


## What is next?

- you can experiment with acqusition functions:
    - compare expected improvement (`EI`) and lower confidence bound (`LCB`) to probability of improvement (`PI`);
- explore different regressions available https://scikit-optimize.github.io/learning/index.html
- try to shoot yourself in the foot by perfroming naive Bayesian inference on a Neural Network.
- try partial Bayesian Inference with Neural Networks: https://arxiv.org/pdf/1502.05700.pdf
- try REMBO-ing your optimum value: https://arxiv.org/abs/1301.1942

<img src="https://imgix.ranker.com/user_node_img/50066/1001300261/original/there-was-a-big-crew-turnover-in-rambo-iii-photo-u1?w=650&q=50&fm=jpg&fit=crop&crop=faces" width=454 height=193>

In [15]:
# import necessary building blocks
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, InputLayer
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K

In [16]:
MAX_PARAMS = int(1.0e+4)

In [17]:
# Function to create the neural net
def make_net(num_nodes1,num_nodes2,num_nodes3,num_nodes4):
    model = Sequential()
    model.add(InputLayer(input_shape=[X_train.shape[1]]))
    model.add(Dense(num_nodes1, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_nodes2, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_nodes3, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(num_nodes4, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation='softmax'))
    return model

In [18]:

gpu_options = tf.GPUOptions(allow_growth=True)
session= tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
keras.backend.tensorflow_backend.set_session(session)
K.clear_session()


# num_nodes = [10, 10, 10, 10]
# this_net = make_net(num_nodes[0],num_nodes[1],num_nodes[2],num_nodes[3])

# this_net.summary()
# this_net.compile(loss='mse', optimizer=keras.optimizers.adamax(lr=5e-3))
# this_net.fit(X_train, y_train, epochs=1,batch_size=32)
# predictions = this_net.predict(X_train)
# print(np.unique(predictions))

/mnt/mlhep2018/pyenv/versions/3.6.6/envs/mlhep/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [19]:
# Function to make the NN with the given parameters

def my_target_function(num_nodes, X_train=X_train, y_train=y_train, X_score=X_test, y_score=y_test):
#     from xgboost import XGBClassifier
    from sklearn.metrics import roc_auc_score
    K.clear_session()
    
#     xgb = XGBClassifier(
#         max_depth=3, 
#         n_estimators=50,
#         learning_rate=np.exp(log_learning_rate),
#         objective='binary:logistic',
#         n_jobs=20,
#         reg_lambda=0.0
#     )

    this_net = make_net(num_nodes[0],num_nodes[1],num_nodes[2],num_nodes[3])
    if this_net.count_params() > MAX_PARAMS:
        return 0.5
    else:
        this_net.summary()
        this_net.compile(loss='mse', optimizer=keras.optimizers.adamax(lr=5e-3))
        this_net.fit(X_train, y_train, epochs=10,batch_size=32)
        predictions = this_net.predict(X_score)
        print(predictions)
#         raise
        return 1 - roc_auc_score(y_score, predictions)

In [20]:
my_dimensions = [
    ## num_nodes range
    (10,50),
    (10,50),
    (10,50),
    (10,50)
]

# dimensions_4 =[
#     ### max_depth
#     (1.0, 20.0),
    
#     ### n_estimators
#     (1.0, 500.0),
    
#     ### log_learning rate
#     (np.log(1.0e-3), np.log(1.0)),
    
#     ### l2 reg
#     (0.0, 1.0)
# ]

In [21]:
my_net_optimizer_struct = Optimizer(
    dimensions=my_dimensions,
#     base_estimator=GaussianProcessRegressor(
#         kernel=RBF(length_scale_bounds=[1.0e-3, 1.0e+3]) +\
#             WhiteKernel(noise_level=1.0e-5, noise_level_bounds=[1.0e-6, 1.0e-2])
#     ),
    base_estimator=GaussianProcessRegressor(
        kernel=Matern(nu=1.5) + WhiteKernel(
            noise_level_bounds=(1.0e-3,1.0e-1))),
    n_initial_points=3,
    acq_func='LCB'
)

In [22]:
for i in range(50):
    x = my_net_optimizer_struct.ask()
    print(x)
    f = my_target_function(x, X_train, y_train, X_val, y_val)
    
    my_net_optimizer_struct.tell(x, f)
    
    plot_convergence(my_net_optimizer_struct)

[14, 23, 29, 13]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 14)                406       
_________________________________________________________________
dropout_1 (Dropout)          (None, 14)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 23)                345       
_________________________________________________________________
dropout_2 (Dropout)          (None, 23)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 29)                696       
_________________________________________________________________
dropout_3 (Dropout)          (None, 29)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 13)                390 

NotFoundError: FeedInputs: unable to find feed output input_1:0